In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from func import show_outliers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


In [32]:
raw_DF=pd.read_csv('brfss2013.csv', encoding='ISO-8859-1', low_memory=False, index_col=0)
raw_DF.columns

Index(['X_state', 'fmonth', 'idate', 'imonth', 'iday', 'iyear', 'dispcode',
       'seqno', 'X_psu', 'ctelenum',
       ...
       'X_pastae1', 'X_lmtact1', 'X_lmtwrk1', 'X_lmtscl1', 'X_rfseat2',
       'X_rfseat3', 'X_flshot6', 'X_pneumo2', 'X_aidtst3', 'X_age80'],
      dtype='object', length=330)

In [33]:
raw_DF['X_cholchk'].mode()

0    Had cholesterol checked in past 5 years
Name: X_cholchk, dtype: object

In [18]:
# 결측치 처리
for col in raw_DF.columns:
    # print(raw_DF[col].dtype)
    if raw_DF[col].dtype=='int' or raw_DF[col].dtype=='float64':
        raw_DF[col].fillna(raw_DF[col].median(), inplace=True)
    elif raw_DF[col].dtype=='object':
        raw_DF[col].fillna(raw_DF[col].mode(dropna=True), inplace=True)

In [30]:
raw_DF['X_flshot6']=raw_DF['X_flshot6'].fillna(raw_DF['X_flshot6'].mode(), inplace=True)
raw_DF['X_flshot6']

1         None
2         None
3         None
4         None
5         None
          ... 
491771    None
491772    None
491773    None
491774    None
491775    None
Name: X_flshot6, Length: 491775, dtype: object

In [29]:
raw_DF['X_flshot6'].mode(dropna=True)

0    Yes
Name: X_flshot6, dtype: object

In [25]:
raw_DF.dropna(how='all',axis=1)

,X_state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,X_psu,ctelenum,...,X_pastae1,X_lmtact1,X_lmtwrk1,X_lmtscl1,X_rfseat2,X_rfseat3,X_flshot6,X_pneumo2,X_aidtst3,X_age80
1,Alabama,January,1092013.0,January,9.0,2013.0,Completed interview,2.013001e+09,2.013001e+09,Yes,...,Did not meet both guidelines,Told have arthritis and have limited usual act...,Told have arthritis and have limited work,Told have arthritis and social activities limi...,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,No,60.0
2,Alabama,January,1192013.0,January,19.0,2013.0,Completed interview,2.013001e+09,2.013001e+09,Yes,...,Did not meet both guidelines,Not told they have arthritis,Not told they have arthritis,Not told they have arthritis,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,Yes,50.0
3,Alabama,January,1192013.0,January,19.0,2013.0,Completed interview,2.013001e+09,2.013001e+09,Yes,...,Did not meet both guidelines,Told have arthritis and have limited usual act...,Told have arthritis and have limited work,Told have arthritis and social activities limi...,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,Yes,55.0
4,Alabama,January,1112013.0,January,11.0,2013.0,Completed interview,2.013001e+09,2.013001e+09,Yes,...,Did not meet both guidelines,Not told they have arthritis,Not told they have arthritis,Not told they have arthritis,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,No,64.0
5,Alabama,February,2062013.0,February,6.0,2013.0,Completed interview,2.013001e+09,2.013001e+09,Yes,...,Did not meet both guidelines,Not told they have arthritis,Not told they have arthritis,Not told they have arthritis,Always or almost always wear seat belt,Always wear seat belt,No,No,No,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491771,Puerto Rico,June,6212013.0,June,21.0,2013.0,Completed interview,2.013005e+09,2.013005e+09,Yes,...,Did not meet both guidelines,Told have arthritis and no limited usual activ...,Told have arthritis and no limited work,Told have arthritis and social activities limi...,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,No,58.0
491772,Puerto Rico,June,6222013.0,June,22.0,2013.0,Completed interview,2.013005e+09,2.013005e+09,Yes,...,Did not meet both guidelines,Told have arthritis and have limited usual act...,Told have arthritis and have limited work,Told have arthritis and social activities not ...,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,No,62.0
491773,Puerto Rico,July,8062013.0,August,6.0,2013.0,Partially completed interview,2.013005e+09,2.013005e+09,Yes,...,Did not meet both guidelines,Not told they have arthritis,Not told they have arthritis,Not told they have arthritis,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,Yes,37.0
491774,Puerto Rico,July,8052013.0,August,5.0,2013.0,Completed interview,2.013005e+09,2.013005e+09,Yes,...,Did not meet both guidelines,Not told they have arthritis,Not told they have arthritis,Not told they have arthritis,Always or almost always wear seat belt,Always wear seat belt,NaN,NaN,Yes,30.0


In [11]:
import torch
import torch.nn
import torch.nn.functional as F
from torchmetrics.classification import F1Score, BinaryF1Score, Accuracy
from torchinfo import summary

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from get_dataset import *
from get_model import *
from get_train_model import *
from func import Torch_proccesing

In [20]:
clf_TS= Torch_proccesing(raw_DF)
clf_TS.feature=clf_TS.data.drop(columns='genhlth')
clf_TS.target= clf_TS.data['genhlth']


In [21]:
X_train, X_test, Y_train, Y_test= train_test_split(clf_TS.feature, clf_TS.target, 
                                                    random_state=123,stratify=clf_TS.target)
X_train, X_val, Y_train, Y_val=train_test_split(X_train, Y_train,
                                                random_state=123,
                                                stratify=Y_train)

ValueError: Input contains NaN

In [ ]:
clf= RandomForestClassifier(verbose=2)
clf.fit()

X_state  fmonth  idate  imonth  iday   iyear  dispcode  seqno  X_psu  ctelenum  pvtresd1  colghous  stateres  cellfon3  ladult  numadult  nummen  numwomen  genhlth  physhlth  menthlth  poorhlth  hlthpln1  persdoc2  medcost  checkup1  sleptim1  bphigh4  bpmeds  bloodcho  cholchk  toldhi2  cvdinfr4  cvdcrhd4  cvdstrk3  asthma3  asthnow  chcscncr  chcocncr  chccopd1  havarth3  addepev2  chckidny  diabete3  veteran3  marital  children  educa  employ1  income2  weight2  height3  numhhol2  numphon2  cpdemo1  cpdemo4  internet  renthom1  sex    pregnant  qlactlm2  useequip  blind  decide  diffwalk  diffdres  diffalon  smoke100  smokday2  stopsmk2  lastsmk2  usenow3  alcday5  avedrnk2  drnk3ge5  maxdrnks  fruitju1  fruit1  fvbeans  fvgreen  fvorang  vegetab1  exerany2  exract11  exeroft1  exerhmm1  exract21  exeroft2  exerhmm2  strength  lmtjoin3  arthdis2  arthsocl  joinpain  seatbelt  flushot6  flshtmy2  tetanus  pneuvac3  hivtst6  hivtstd3  whrtst10  pdiabtst  prediab1  diabage2  insulin  b

In [7]:
life_DF=life_DF[9:]
life_col= life_DF.columns
for co in life_col:
    if life_DF[co].dtype=='object' and co != 'genhlth':
        life_DF[co].bfill()
        life_DF[co]=LabelEncoder().fit_transform(life_DF[co])

In [8]:
life_DF['genhlth']=life_DF['genhlth'].bfill()
life_DF['genhlth'].replace({'Poor':0, 'Fair':1, 'Good':2, 'Very good':3, 'Excellent':4},
                                   inplace=True)

[2]-1 학습 모델 설계


[2]-2 데이터셋 설계

In [10]:
life= Torch_proccesing(life_DF)

In [11]:
# life_T.data['genhlth'].replace({'Poor':0, 'Fair':1, 'Good':2, 'Very good':3, 'Excellent':4},
#                                    inplace=True)

In [12]:
# employ -> 원핫 인코딩
# em= ['Unable to work', 'Out of work for 1 year or more', 'A student', 'Out of work for less than 1 year',
#      'A homemaker', 'Self-employed', 'Retired', 'Employed for wages']
# value= [1,2,3,4,5,6,7,8]
# life_T.data=pd.get_dummies(life_T.data, columns=['employ'], drop_first=True, sparse=False)

In [13]:
life.data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491766 entries, 10 to 491775
Columns: 330 entries, X_state to X_age80
dtypes: float64(99), int32(230), int64(1)
memory usage: 810.4 MB


In [14]:
for col in life.data.columns:
    print(col, end=' ')

X_state fmonth idate imonth iday iyear dispcode seqno X_psu ctelenum pvtresd1 colghous stateres cellfon3 ladult numadult nummen numwomen genhlth physhlth menthlth poorhlth hlthpln1 persdoc2 medcost checkup1 sleptim1 bphigh4 bpmeds bloodcho cholchk toldhi2 cvdinfr4 cvdcrhd4 cvdstrk3 asthma3 asthnow chcscncr chcocncr chccopd1 havarth3 addepev2 chckidny diabete3 veteran3 marital children educa employ1 income2 weight2 height3 numhhol2 numphon2 cpdemo1 cpdemo4 internet renthom1 sex pregnant qlactlm2 useequip blind decide diffwalk diffdres diffalon smoke100 smokday2 stopsmk2 lastsmk2 usenow3 alcday5 avedrnk2 drnk3ge5 maxdrnks fruitju1 fruit1 fvbeans fvgreen fvorang vegetab1 exerany2 exract11 exeroft1 exerhmm1 exract21 exeroft2 exerhmm2 strength lmtjoin3 arthdis2 arthsocl joinpain seatbelt flushot6 flshtmy2 tetanus pneuvac3 hivtst6 hivtstd3 whrtst10 pdiabtst prediab1 diabage2 insulin bldsugar feetchk2 doctdiab chkhemo3 feetchk eyeexam diabeye diabedu painact2 qlmentl2 qlstres2 qlhlth2 medicar

In [15]:

# 피쳐와 타겟 선정
life.feature=life.data.drop(columns='genhlth')
life.target= life.data['genhlth']

In [16]:
# 테이터 분리
life.split(val=True, random_state=921, stratify=True)

train: (276618, 329),(276618,)
test: (122942, 329),(122942,)
val: (92206, 329),(92206,)
train: (368824, 329),(368824,)
test: (122942, 329),(122942,)


In [17]:
from torch.utils.data import DataLoader
import torch.optim as optim

In [18]:
# 스케일링 -> minmax
scaler= MinMaxScaler()
scaler.fit(life.X_train)
X_train_scaled=scaler.transform(life.X_train)
X_test_scaled= scaler.transform(life.X_test)
X_val_scaled= scaler.transform(life.X_val)

In [19]:
X_train_scaled=pd.DataFrame(X_train_scaled, columns=life.feature.columns)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=life.feature.columns)
X_val_scaled=pd.DataFrame(X_val_scaled, columns=life.feature.columns)

In [20]:
# 데이터셋, 로더 생성
trainDS= Custom_Dataset(featureDF=X_train_scaled, targetDF=life.Y_train.to_frame())
testDS= Custom_Dataset(featureDF=X_test_scaled, targetDF=life.Y_test.to_frame())
valDS= Custom_Dataset(featureDF=X_val_scaled, targetDF=life.Y_val.to_frame())

trainDL=DataLoader(trainDS, batch_size=64)
testDL= DataLoader(testDS, batch_size=64)
valDL= DataLoader(valDS, batch_size=64)

[3] 학습준비

- 학습횟수: EPOCH
- 배치크기: 64
- 위치 지정: DEVICE
- 학습률: LR

In [21]:
len(trainDL)

5763

##### 학습 모델
- 모델 이름: health
- 사용 피쳐: 21개
- 사용 타겟: 전반적인 건강 정도 (점수로 나타 낼 수 있으면 좋을듯?)
- 사용 프레임워크: torch
- 사용 알고리즘: DNN
- 사용 최적화함수: nn.Adam
- 입력층: 25-> 150
- 은닉층: 150-> 120, 120-> 100, 100->80
- 출력층 80-> 5

In [22]:
# 설정값
EPOCH=100
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
LR= 1
torch.manual_seed(1)

In [23]:
class make_model(nn.Module):
    '''
    커스텀 모델을 만드는 함수
    model_type= 'reg'|'binary'|'mclf'
    은닉층 수= 리스트 수-1
    '''
    def __init__(self, in_in, out_out, hidden: list) -> None:
        super().__init__()


        self.in_layer= nn.Linear(in_in, hidden[0])
        self.h1_layer=nn.Linear(hidden[0], hidden[1])
        self.h2_layer=nn.Linear(hidden[1], hidden[2])
        self.h3_layer=nn.Linear(hidden[2], hidden[3])
        self.out_layer= nn.Linear(hidden[-1], out_out)
    
    def forward(self, input_data):
        
        y= self.in_layer(input_data)
        y= F.relu(y)

        
        y=self.h1_layer(y)
        y=F.relu(y)
        
        y=self.h2_layer(y)
        y=F.relu(y)

        y=self.h3_layer(y)
        y=F.relu(y)

        y=self.out_layer(y)
        
        
        return y

In [24]:
health=make_model(in_in=329, out_out=5, hidden=[150,120,100,80])

In [25]:
# 최적화 인스턴스
optimizer= optim.SGD(health.parameters(), lr=LR)

# 저장
import os
# 경로
SAVE_PATH= '../models/health/'
# 이름
SAVE_FILE='model_train_all.pth'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)       #하위 폴더까지 생성

In [26]:
for f, t in trainDL:
    # print(f, t)
    print(f.shape, t.shape)
    break

torch.Size([64, 329]) torch.Size([64, 1])


In [27]:
life_DF.describe()

,X_state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,X_psu,ctelenum,...,X_pastae1,X_lmtact1,X_lmtwrk1,X_lmtscl1,X_rfseat2,X_rfseat3,X_flshot6,X_pneumo2,X_aidtst3,X_age80
count,491766.000000,491766.000000,4.917640e+05,491766.000000,491763.000000,491761.000000,491766.000000,4.917630e+05,4.917640e+05,491766.000000,...,491766.000000,491766.000000,491766.000000,491766.000000,491766.000000,491766.000000,491766.000000,491766.000000,491766.000000,491755.000000
mean,27.544383,5.441436,6.672334e+06,5.476015,13.749635,2013.011554,0.119077,2.012998e+09,2.012994e+09,0.000083,...,0.444943,0.558601,0.620206,0.846531,0.213020,0.283151,1.582192,1.621035,0.490807,54.773696
std,14.451279,3.437338,3.412144e+06,3.464415,8.253900,0.106869,0.323911,4.059589e+06,4.971950e+06,0.009131,...,0.733290,0.860231,0.929449,1.294700,0.570985,0.601123,0.688566,0.642528,0.686988,17.032090
min,0.000000,0.000000,1.000000e+00,0.000000,1.000000,2013.000000,0.000000,1.000000e+00,1.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,3.000000,4.022013e+06,2.000000,7.000000,2013.000000,0.000000,2.013002e+09,2.013002e+09,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,42.000000
50%,26.000000,5.000000,7.022013e+06,5.000000,13.000000,2013.000000,0.000000,2.013005e+09,2.013005e+09,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,57.000000
75%,39.000000,8.000000,1.001201e+07,8.000000,20.000000,2013.000000,0.000000,2.013008e+09,2.013008e+09,0.000000,...,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,2.000000,2.000000,1.000000,68.000000
max,54.000000,12.000000,1.231201e+07,12.000000,31.000000,2014.000000,2.000000,2.013034e+09,2.013034e+09,1.000000,...,2.000000,3.000000,3.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,80.000000


In [28]:
life.Y_test.value_counts()

genhlth
3    40199
2    37796
4    21277
1    16695
0     6975
Name: count, dtype: int64

In [29]:
# health=Custom_model(in_in=25, out_out=5, hidden=[150,120,100,80], af=F.relu, model_type='multiclass')
# health

In [30]:
model_training(health, trainDL=trainDL, testDL=valDL, optimizer=optimizer, epoch=100,
               break_param='score', type='muticlass', SAVE_FILE=SAVE_FILE, SAVE_PATH=SAVE_PATH, save_type='all',
               numcls=5, LIMIT=15, optim_type='score')

1/100


KeyboardInterrupt: 

In [33]:
model=health
trainDL=trainDL; testDL=valDL;  optimizer=optimizer;  epoch=100
break_param='score'; mtype='muticlass'; SAVE_FILE=SAVE_FILE; SAVE_PATH=SAVE_PATH; save_type='all'
numcls=5; LIMIT=15; optim_type='score'




scheduler= lr_scheduler.ReduceLROnPlateau(optimizer, patience=LIMIT, mode='max')
EPOCH=100
# 손실, 평가값 저장
LOSS_HISTORY, SCORE_HISTORY= [[],[]], [[],[]]
for ep in range(EPOCH):
    print(f'{ep+1}/{EPOCH}')
    model.train()
    loss_total, score_total= 0,0
    loss_val_total, score_val_total=0,0

    for train_feature, train_target in trainDL:
        # 학습
        pre_y=model(train_feature)
        # 손실
        if mtype=='reg':
            Lossfunc=MeanSquaredError()
            Scorefunc=R2Score()
        elif mtype=='binary':
            Lossfunc= nn.BCELoss()
            Scorefunc=F1Score(task='binary', num_classes=numcls)
        elif mtype=='muticlass':
            Lossfunc=nn.CrossEntropyLoss()
            Scorefunc=Accuracy(task='MULTICLASS',num_classes=5)

        loss= Lossfunc(pre_y, train_target.reshape(-1).long() if mtype=='muticlass' else train_target)
        loss_total+=loss.item()

        # 평가
        score= Scorefunc(pre_y, train_target.reshape(-1) if mtype=='muticlass' else train_target if mtype=='reg' else train_target)
        score_total+=score.item()
        # 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 검증

    model.eval()
    with torch.no_grad():
        for val_feature, val_target in testDL:
            # 학습
            pre_val= model(val_feature)

            # 손실
            loss= Lossfunc(pre_val, val_target.reshape(-1).long() if mtype=='muticlass' else val_target if mtype=='reg' else val_target)
            loss_val_total+=loss.item()

            # 평가
            score= Scorefunc(pre_val, val_target.reshape(-1) if mtype=='muticlass' else val_target)
            score_val_total+=score.item()

    
    # 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))
    print(f'Train\n Loss: {loss_total/len(trainDL)}\n Score: {score_total/len(trainDL)}')

    LOSS_HISTORY[1].append(loss_val_total/len(testDL))
    SCORE_HISTORY[1].append(score_val_total/len(testDL))
    print(f'Val\n Loss: {loss_val_total/len(testDL)}\n Score: {score_val_total/len(testDL)}')

    # 성능이 좋은 학습 가중치 저장
    if save_type:
        if save_type=='all':
            save_type= model
        elif save_type=='param':
            save_type=model.state_dict()
        if len(SCORE_HISTORY[1]) == 1: 
        #첫번째는 무조건 저장
            torch.save(save_type, SAVE_PATH+SAVE_FILE)  
            
        else:
            if SCORE_HISTORY[1][-1]> max(SCORE_HISTORY[1][:-1]): # 자신을 제외한 최대점수값과 비교
                torch.save(save_type, SAVE_PATH+SAVE_FILE) 
                    
    else: pass

    
    # 학습 진행 모니터링 (검증 데이터 개선이 되지 않았을때 누적 ->  평가, 손실 중 지표 하나 선택)
    # 최적화 스케쥴러 인스턴스 업데이트
    scheduler.step(score_val_total/len(testDL))
    # print(f'scheduler.num_bad_epochs: {scheduler.num_bad_epochs}', end=' ') #보여주기용
    # print(f'scheduler.patience: {scheduler.patience}')
    # 손실 감소 (또는 성능 개선)이 안되는 경우 조기종료
    if scheduler.num_bad_epochs== scheduler.patience:
        print(f'{scheduler.patience} EPOCH 성능 개선이 없어서 조기종료함')
        break

1/100
Train
 Loss: nan
 Score: 0.057223110483500685
Val
 Loss: nan
 Score: 0.057097746885273706
2/100
Train
 Loss: nan
 Score: 0.057223110483500685
Val
 Loss: nan
 Score: 0.057097746885273706
3/100


KeyboardInterrupt: 

In [137]:
life.X_train.corr()

MemoryError: Unable to allocate 926. MiB for an array with shape (329, 368824) and data type float64